In [1]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from sklearn.model_selection import train_test_split

In [2]:
label_name = "math"

dataset = pd.read_csv(f"../dataset/{label_name}_dataset.csv")
dataset = pd.get_dummies(dataset)

train, test = train_test_split(dataset, test_size=0.5, random_state=42, shuffle=True)

dataset.head()

,branch-misses_FEATURE_CONFIG,branch-misses_INTERCEPT,branch-misses_R-VAL,branches_FEATURE_CONFIG,branches_INTERCEPT,branches_R-VAL,context-switches_FEATURE_CONFIG,context-switches_INTERCEPT,context-switches_R-VAL,cpu-migrations_FEATURE_CONFIG,...,stalled-cycles-frontend_FEATURE_TYPE_LOGLOG_POLYNOMIAL,stalled-cycles-frontend_FEATURE_TYPE_LOG_POLYNOMIAL,stalled-cycles-frontend_FEATURE_TYPE_POLYNOMIAL,stalled-cycles-frontend_FEATURE_TYPE_POWER,task-clock_FEATURE_TYPE_FACTORIAL,task-clock_FEATURE_TYPE_FRACTIONAL_POWER,task-clock_FEATURE_TYPE_LOGLOG_POLYNOMIAL,task-clock_FEATURE_TYPE_LOG_POLYNOMIAL,task-clock_FEATURE_TYPE_POLYNOMIAL,task-clock_FEATURE_TYPE_POWER
0,0.0,12176.278640,5.274472,1.0,354489.841881,31.023435,0,0.0,0.0,0,...,0,1,0,0,0,0,0,0,1,0
1,0.8,12300.866448,140.866882,0.1,360567.282593,278.416553,0,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.0,12396.648839,38.928139,1.0,358483.822954,81.019933,0,0.0,0.0,0,...,0,0,1,0,0,0,0,0,1,0
3,4.0,12358.582719,-0.000011,1.0,355754.502857,256.760672,0,0.0,0.0,0,...,0,0,0,1,0,1,0,0,0,0
4,1.0,12313.598463,3.715744,1.0,354852.459189,327.289900,0,0.0,0.0,0,...,0,1,0,0,0,1,0,0,0,0


In [3]:
dataset_features = train.copy()
dataset_labels = train.copy().pop('label')

In [4]:
dataset_features = np.array(dataset_features)

In [5]:
normalize = preprocessing.Normalization()
normalize.adapt(dataset_features)

model = tf.keras.Sequential([
  normalize,
  layers.Dense(1024),
  layers.Dense(512),
  layers.Dense(512),
  layers.Dense(128),
  layers.Dense(128),
  layers.Dense(64),
  layers.Dense(64),
  layers.Dense(32),
  layers.Dense(32),
  layers.Dense(8),
  layers.Dense(8),  
  layers.Dense(4),
  layers.Dense(2),
  layers.Dense(1, activation='sigmoid')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer = tf.optimizers.Adam(),
              metrics=['accuracy'])

2021-10-10 11:52:47.872237: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-10 11:52:47.930583: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [6]:
model.fit(dataset_features, dataset_labels, epochs=50)

Epoch 1/50
90/90 [==============================] - 1s 6ms/step - loss: 0.4733 - accuracy: 0.9671
Epoch 2/50
90/90 [==============================] - 0s 6ms/step - loss: 0.8613 - accuracy: 0.9444
Epoch 3/50
90/90 [==============================] - 0s 5ms/step - loss: 0.2569 - accuracy: 0.9885
Epoch 4/50
90/90 [==============================] - 0s 5ms/step - loss: 1.0487 - accuracy: 0.9860
Epoch 5/50
90/90 [==============================] - 0s 5ms/step - loss: 0.2431 - accuracy: 0.9794
Epoch 6/50
90/90 [==============================] - 0s 5ms/step - loss: 1.1397 - accuracy: 0.9608
Epoch 7/50
90/90 [==============================] - 0s 5ms/step - loss: 3.1164e-05 - accuracy: 1.0000
Epoch 8/50
90/90 [==============================] - 0s 5ms/step - loss: 2.1537e-05 - accuracy: 1.0000
Epoch 9/50
90/90 [==============================] - 0s 5ms/step - loss: 1.6321e-05 - accuracy: 1.0000
Epoch 10/50
90/90 [==============================] - 1s 6ms/step - loss: 1.3067e-05 - accuracy: 1.0000
Epo

In [7]:
train_dataset_features = train.copy()
train_dataset_labels = train.copy().pop('label')

tf.math.confusion_matrix(train_dataset_labels, model.predict(train_dataset_features))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2460,    0],
       [  25,  374]], dtype=int32)>

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

preds = np.array(model.predict(train_dataset_features)).astype(int)

accuracy = accuracy_score(train_dataset_labels.to_numpy(), preds)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(train_dataset_labels.to_numpy(), preds)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(train_dataset_labels.to_numpy(), preds)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(train_dataset_labels.to_numpy(), preds)
print('F1 score: %f' % f1)

Accuracy: 0.991256
Precision: 1.000000
Recall: 0.937343
F1 score: 0.967658


In [9]:
test_dataset_features = test.copy()
test_dataset_labels = test.copy().pop('label')

tf.math.confusion_matrix(test_dataset_labels, model.predict(test_dataset_features))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2382,    2],
       [  38,  437]], dtype=int32)>

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

preds = np.array(model.predict(test_dataset_features)).astype(int)

accuracy = accuracy_score(test_dataset_labels.to_numpy(), preds)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_dataset_labels.to_numpy(), preds)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_dataset_labels.to_numpy(), preds)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(test_dataset_labels.to_numpy(), preds)
print('F1 score: %f' % f1)

Accuracy: 0.986009
Precision: 0.995444
Recall: 0.920000
F1 score: 0.956236
